In [1]:
from bokeh.events import ButtonClick
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, CustomJS, Button, Slider, DataRange1d, LabelSet, RadioButtonGroup, Div
from bokeh.plotting import figure
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.io import output_notebook, show, curdoc
import IPython

‘Quantum physics’ is a term widely used but much less understood. It is a mathematical model first used to describe the behavior of small things in a laboratory, which exposed gaps in the preceding theory of **‘classical’** (pre-quantum) physics. Quantum theory explains this behavior and gives a more complete picture of our universe. This previously unexplained behavior can be used to perform certain computations that was previously believed impossible. This is **quantum computing.** Quantum computing distills the core concepts from quantum physics into their simplest forms, stripping away the complications of the physical world.

## Review of Classical Probability

If a fair coin is tossed, there is a 50% chance of Heads, and a 50% chance of Tails. Around 500 to 1000 tosses should be enough to get reliable results.

In [7]:
data_labels = ["Heads", "Tails"]
initial_counts = [0] * 2
initial_labels = ["0.00%"] * 2

source = ColumnDataSource(data=dict(x=data_labels, top=initial_counts, top_percent=initial_labels))

plot = figure(
    plot_height=500, 
    plot_width=500,
    sizing_mode="scale_width",
    tools="reset",
    x_range=data_labels,
    y_range=DataRange1d(start=0, range_padding = 20, range_padding_units='absolute'),
    y_axis_label="Counts"
)
plot.vbar(top='top',x='x',width=0.5,source=source, color="#BE95FF")
labels = LabelSet(x='x', y='top', text='top_percent', level='glyph', x_offset=-20, y_offset=12, source=source, render_mode='canvas')
plot.add_layout(labels)

tossbtn = Button(label="Toss Coin")
repeat_slider = Slider(title="No. of Coins", start=1, end=50, step=1, value=0)
radio_label = Div(text="Initial State:")
init_state_radio = RadioButtonGroup(name="Initial State", labels=data_labels, active=0)
resetbtn = Button(label="Reset")
results_label = Div(text="Results:")
resultsHTML = Div(text="", style={'overflow-y': 'auto', 'max-height': '250px'})

toss_callback = CustomJS(args=dict(source=source, p=plot, s=repeat_slider, resultsHTML=resultsHTML, results_label=results_label), 
    code="""
        for (var i = 0; i <= s.value; ++i) {
            const result = Math.floor(Math.random() * 2);
            source.data.top[result] += 1;
            resultsHTML.text += [' H ', ' T '][result]
        }
        const total_tosses = source.data.top[0] + source.data.top[1];
        results_label.text = "Results (" + total_tosses + " tosses):";

        for (var i = 0; i<2; ++i){
            const frac = source.data.top[i] / source.data.top.reduce((a, b) => a + b, 0);
            source.data.top_percent[i] = (frac*100).toFixed(2) + "%";
        }
        if (Math.max(...source.data.top) > 22) {
            p.y_range.range_padding_units = 'percent';
            p.y_range.range_padding = 1;
        } else {
            p.y_range.range_padding_units = 'absolute';
            p.y_range.range_padding = 30 - Math.max(...source.data.top);
        };
        source.change.emit();
    """
)

slider_callback = CustomJS(args=dict(tossbtn=tossbtn), 
    code="""
        const repeats = cb_obj.value;
        if (repeats == 1) {
            tossbtn.label = "Toss Coin";
        } else {
            tossbtn.label = "Toss " + repeats + " Coins";
        };
    """
)

reset_callback = CustomJS(args=dict(source=source, resultsHTML=resultsHTML, results_label=results_label), 
    code="""
        source.data.top = [0,0];
        source.data.top_percent = ['0.00%', '0.00%'];
        source.change.emit();
        resultsHTML.text = "";
        results_label.text = "Results:";
    """
)

tossbtn.js_on_event(ButtonClick, toss_callback)
repeat_slider.js_on_change('value', slider_callback)
resetbtn.js_on_event(ButtonClick, reset_callback)

control_panel = column(tossbtn, repeat_slider, radio_label, init_state_radio, resetbtn, results_label, resultsHTML)

layout = row(plot, control_panel)

html_repr = file_html(layout, CDN)
IPython.display.HTML(html_repr)

<iframe sandbox='allow-scripts allow-forms' style='display:block; margin:0px;' frameborder='0' srcdoc='<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Bokeh Application</title>
    <script type="text/javascript" src="https://cdn.bokeh.org/bokeh/release/bokeh-2.4.3.min.js"></script>
    <script type="text/javascript" src="https://cdn.bokeh.org/bokeh/release/bokeh-widgets-2.4.3.min.js"></script>
    <script type="text/javascript">
        Bokeh.set_log_level("info");
    </script>
  </head>
  <body>
    <div class="bk-root" id="4713c925-b5e8-4dd7-8e15-0eeb9d8ea183" data-root-id="1846"></div>
  
    <script type="application/json" id="1967">
      {"9213a5f9-9cde-45f7-8c84-36be2210c93b":{"defs":[],"roots":{"references":[{"attributes":{"text":"Initial State:"},"id":"1837","type":"Div"},{"attributes":{"icon":null,"js_event_callbacks":{"button_click":[{"id":"1844"}]},"label":"Reset"},"id":"1839","type":"Button"},{"attributes":{"style":{"max-height":"250px","ov